Now, we want to compare the ages of death for these 9 individuals with the median age of death for those in the same municipality.

Again, we start by importing the data into Python using a Pandas library on the median age of death for this period. In particular, the Western Pennsylvania Regional Data Center has tracked the median age at death in Allegheny County between 2011 and 2015 and made that available at "https://data.wprdc.org/dataset/median-age-death".

Fortunately, this already came as CSV file, and we could then upload it to Jupyter Hub. Reading the file into Python results in the following table showing 131 Allegheny County municipalities and their respective data.

In [2]:
import pandas as pd
wprdc_data = pd.read_csv("files/munimedianage20112015.csv")
wprdc_data.head()

,Municipality,BLACKdeaths,BLACK MD AGE AT DEATH,WHITEdeaths,WHITE MD AGE AT DEATH,TOTALdeaths,TOTAL MD AGE AT DEATH
0,Haysville Borough,0,0.0,5,77.5,5,77.5
1,Trafford Borough,0,0.0,5,80.0,5,80.0
2,Ben Avon Heights Borough,0,0.0,10,86.7,10,86.7
3,Glenfield Borough,0,0.0,16,82.5,16,82.5
4,Glen Osborne Borough,0,0.0,19,78.6,19,78.6


Looking through the municipalities here and comparing to our list of shooting victims, we can see we will have a few issues merging these datasets. First, there's a pesky misspelling of "Pittsburg" in Shawn Evans' listing. Let's fix that now with a simple "replace" modification.

In [ ]:
allegheny_violence_sic = allegheny_violence.replace("Pittsburg","Pittsburgh")
allegheny_violence_sic.head()

Another issue is that the WPRDC data does not include a listing for the Knoxville neighborhood. However, the WPRDC data does include Mount Oliver, which is adjacent to Knoxville, and so we will substitute Knoxville into the WPRDC dataframe to get the median age of death for the adjacent neighborhood.

In [10]:
wprdc_sub = wprdc_data.replace("Mount Oliver Borough","Knoxville")
wprdc_sub.head()

,Municipality,BLACKdeaths,BLACK MD AGE AT DEATH,WHITEdeaths,WHITE MD AGE AT DEATH,TOTALdeaths,TOTAL MD AGE AT DEATH
0,Haysville Borough,0,0.0,5,77.5,5,77.5
1,Trafford Borough,0,0.0,5,80.0,5,80.0
2,Ben Avon Heights Borough,0,0.0,10,86.7,10,86.7
3,Glenfield Borough,0,0.0,16,82.5,16,82.5
4,Glen Osborne Borough,0,0.0,19,78.6,19,78.6


The last, more serious issue, is that the "City" designation in this table doesn't include Township or Borough identifiers that the WPRDC data does. We could just do another replace, but we may as well do something a little more complicated.

In [ ]:
list_mediandeath = []
for municipality in allegheny_violence_sic["City"] :
    if allegheny_violence_sic.loc[allegheny_violence_sic.City == municipality, 'Victim's race'].values[0] == "Black" :
        list_mediandeath.append(wprdc.loc[wprdc_sub.Municipality == wprdc_sub["Municipality"].str.find(municipality), "BLACK MD AGE AT DEATH"].values[0])
    else if allegheny_violence_sic.loc[allegheny_violence_sic.City == municipality, 'Victim's race'].values[0] == "White" :
        list_mediandeath.append(wprdc.loc[wprdc_sub.Municipality == wprdc_sub["Municipality"].str.find(municipality), "WHITE MD AGE AT DEATH"].values[0])
allegheny_violence_sic.insert(2,"Median Age of Death in Municipality by Race",list_mediandeath)

What we did here is to create a new Pandas series of just the median age of death for each municipality broken down by victim race. We started with an empty list and then created a for-loop to search through each City entry in the spell-checked Allegheny Violence dataframe.

From there, we searched for the row corresponding to each City and column corresponding to the victim's race to give us the race of each victim. Then, we could do a similar search of the WPRDC data to find the row corresponding to the City and then the column corresponding to the median age of death, specific to the race we found earlier. That value was appended to the initial list, and that list was then added as a Pandas series into the original dataframe.

Last thing to do for this part is to figure out the difference between each victim's age of death and the median for their municipality before comparing the difference between each race. To do that, we subtract the age of the victim from our new median for their municipality. Positive values will show the victim(s) was/were younger than the median, and negative values will indicate the victim(s) was/were older than the median.

In [ ]:
allegheny_violence_sic["Difference (Median-Actual)"] = allegheny_violence_sic["Median Age of Death in Municipality by Race"] - allegheny_violence_sic["Victim's age"]
allegheny_violence_sic

And then we can compute the average difference, separated by race, using our earlier mask.

In [ ]:
w_victims_avg = allegheny_violence_sic[w_mask]["Victim's age"].mean()
w_victims_avg

In [ ]:
b_victims_avg = allegheny_violence_sic[b_mask]["Victim's age"].mean()
b_victims_avg

As we can see, (black/white) victims have a higher difference in death age compared to their neighbors of the same race.